In [266]:
import pandas as pd
import os
import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import matplotlib.pyplot as plt
import re
from wordcloud import WordCloud, STOPWORDS

#pip install -U pip setuptools wheel
#pip install -U spacy
#python -m spacy download en_core_web_sm
import spacy
nlp = spacy.load('en_core_web_sm')

pd.set_option('display.max_colwidth', 0)
#print(os.getcwd())

In [267]:
#code to toggle the code cell blocks
HTML('''
<script>
codeShow=true;
function codeToggle() {
    if(codeShow) { $('div.input').hide(); } else { $('div.input').show(); }
    codeShow = !codeShow;
}
$(document).ready(codeToggle)
</script>
<form action="javascript:codeToggle();"><input type="submit" value="..."/></form>
''')

In [268]:
new_api_uri = 'https://newsapi.org/v2/everything?language=en&pageSize=100&q={0}&apiKey={1}'
api_key = '6f32d0305ef44edfb64fc31dc6432a04'

In [269]:
def search_news(x):
    search='x'
    news_api_query = new_api_uri.format(search, api_key)
    response_df=pd.read_json(news_api_query)
    raw_df = pd.DataFrame(response_df['articles'].tolist())
    raw_df['source_name'] = pd.json_normalize(raw_df['source'])['name']
    raw_df['news_text'] = raw_df['title'] + ' ' + raw_df['description']
    all_news_text = '.\n '.join(raw_df['news_text'])
    
    plot_wc(all_news_text.lower())
    detect_entities_in_newstext(all_news_text)
    
    return None

In [270]:
def detect_entities_in_newstext(text):
    doc = nlp(text)
    
    ents_dict = {}
    for ent in doc.ents:
        # create a dict for the entity type as key and set of entity names
        ents_dict[ent.label_] = ents_dict[ent.label_] if ent.label_ in ents_dict.keys() else set()
        ents_dict[ent.label_].add(ent.text)
    
    data_list = []
    for k,v in ents_dict.items():
        data_list.append([k, ', '.join(v)])
        
    ent_df = pd.DataFrame(data_list, columns=['Entity', 'Name'])
    ent_df.set_index('Entity', inplace=True)
    display(ent_df)
    spacy.displacy.render(doc, style='ent', jupyter=True)

In [271]:
def plot_wc(text):    
    wordcloud = WordCloud(background_color='white', width=1600, height=600, max_words=100, relative_scaling=0.5,
                     stopwords=STOPWORDS, normalize_plurals=True).generate(text)
    plt.figure(figsize=(20,10))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.show()

In [272]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [273]:
text_input = widgets.Text(value='', placeholder='', description='Search topic', disabled=False)
im = interact_manual(search_news, x=text_input);
im.widget.children[1].description = 'Run Search'
display(im)

interactive(children=(Text(value='', description='Search topic', placeholder=''), Button(description='Run Inte…

<function __main__.search_news(x)>